In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./data/cleaned_recipes.csv')
data.head()

,Name,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,Ingredients,Allergens
0,Low-Fat Berry Blue Frozen Dessert,"c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,"blueberries, granulated sugar, vanilla yogurt,...",Dairy
1,Biryani,"c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,"saffron, milk, hot green chili peppers, onions...","Dairy, Dairy, Eggs"
2,Best Lemonade,"c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,"sugar, lemons, rind of, lemon, zest of, fresh ...",NaN
3,Cabbage Soup,"c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"plain tomato juice, cabbage, onion, carrots, c...",NaN
4,Best Blackbottom Pie,"c(""graham cracker crumbs"", ""sugar"", ""butter"", ...",437.9,19.3,10.9,94.3,267.6,58.0,1.8,42.5,7.0,"graham cracker crumbs, sugar, butter, sugar, c...","Dairy, Dairy, Dairy"


In [3]:
nutritional_columns = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
                       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent']


In [13]:
def last_order(order_name):
    """
    Fetch nutritional data for the given last ordered food.

    Args:
        order_name (str): The name of the last ordered food.

    Returns:
        pd.Series or None: Nutritional information for the food, or None if not found.
    """
    match = data[data['Name'] == order_name]
    if not match.empty:
        return match[nutritional_columns].iloc[0]
    else:
        print(f"'{order_name}' not found in the data.")
        return None

def food_recommender(input_food, user_allergens=[]):
    """
    Recommends food based on nutritional similarity to a given food, while filtering out allergens.

    Args:
        input_food (str): Name of the input food.
        user_allergens (list): List of allergens to filter out.

    Returns:
        pd.DataFrame: Recommendations with relevant details.
    """
    input_nutritional_data = last_order(input_food)
    if input_nutritional_data is None:
        return None  # Exit if no matching food found.

    scaler = MinMaxScaler()
    input_food_scaled = scaler.fit_transform([input_nutritional_data.values])  # Scale input food data

    # Apply allergen filter
    if user_allergens:
        filtered_data = data[~data['Allergens'].str.contains('|'.join(user_allergens), na=False)]
    else:
        filtered_data = data

    # Scale nutritional data of the filtered dataset
    filtered_food_data = scaler.transform(filtered_data[nutritional_columns])

    # Calculate similarity scores
    similarity_scores = cosine_similarity(input_food_scaled, filtered_food_data)

    # Get top recommendations
    top_indices = similarity_scores[0].argsort()[::-1][:10]
    recommendations = filtered_data.iloc[top_indices]

    # Relevant columns for display
    relevant_columns = ['Name', 'Ingredients', 'Allergens'] + nutritional_columns
    return recommendations[relevant_columns]


In [14]:
# Example usage
recommendations = food_recommender("Chai Tea", ["Soy"])
recommendations.head()
# if recommendations is not None:
#     print(recommendations)

C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


,Name,Ingredients,Allergens,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
461944,Spicy Baked Scotch Eggs,"hard-boiled eggs, breakfast sausage, panko bre...","Gluten, Gluten, Eggs",1093.3,71.2,22.2,1769.7,1318.6,29.7,1.3,6.7,76.4
154033,S's No Cook Peanut Butter Pie,"peanut butter, confectioners' sugar, low-fat c...","Dairy, Dairy, Dairy, Nuts",549.1,40.1,18.3,28.7,357.8,38.2,2.6,32.3,15.0
154035,Salisbury Steak,"ground beef, onion",NaN,198.3,13.0,5.0,57.8,80.7,3.0,0.3,0.6,16.1
154036,Sweet &amp; Sour Watermelon Cucumber Salad,"cider vinegar, water, sugar, poppy seed, green...",Sulphites,87.3,0.7,0.1,0.0,4.3,20.3,0.8,17.6,1.1
154037,Creamy Garlic Pasta With Mushrooms and Prosciutto,"chicken broth, garlic cloves, crushed red pepp...","Dairy, Dairy",612.6,25.4,14.7,79.8,488.0,81.6,12.2,4.8,18.5
